This notebook shows how to use [dvc](https://dvc.org/) [experiments](https://github.com/iterative/dvc/wiki/Experiments) in model development. This example uses the [MNIST](http://yann.lecun.com/exdb/mnist/) data of handwritten digits and builds a classification model to predict the digit (0-9) in each image. The model is built in [pytorch](https://pytorch.org/) as a convolutional neural network with a simplified architecture, which should be able to quickly run on most computers.

### Get started

To get started, clone this repository and navigate to it.

The only other prerequisite is [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/). Once conda is installed, create a virtual environment from the existing `environment.yaml` file and activate it:

```bash
conda env create -f environment.yml
conda activate dvc
```

If you want to run this notebook directly, do so after activating the conda environment.

Finally, initialize dvc and enable the experiments feature:

In [1]:
%%bash
dvc init
dvc config --global core.experiments true


You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


### Establish the pipeline DAG

Before experimenting, a dvc pipeline must be established (see the docs if you are new to dvc). Review the contents of `dvc.yaml` below to see the pipeline.

In [2]:
%%bash
cat dvc.yaml

stages:
  download:
    cmd: python download.py
    deps:
    - download.py
    outs:
    - data/MNIST
  train:
    cmd: python train.py
    deps:
    - data/MNIST
    - train.py
    params:
    - lr
    - weight_decay
    outs:
    - model.pt:
        checkpoint: true
    metrics:
    - metrics.yaml


The download stage gets the data using the `download.py` script. The train stage performs model training and evaluation on the downloaded data using the `train.py` script. The train stage uses the lr and weight_decay metrics defined in `params.yaml`. The model output is saved to `model.pt`, and the metrics are saved to `metrics.yaml`.

Execute the download stage to get the data.

In [3]:
%%bash
dvc repro download

Running stage 'download' with command:
	python download.py
Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock data/.gitignore
Use `dvc push` to send your updates to remote storage.


9920512it [00:02, 4609591.93it/s]                             
 28%|██▊       | 8192/28881 [00:00<00:00, 80144.94it/s]
0it [00:00, ?it/s]
 85%|████████▌ | 1409024/1648877 [00:00<00:00, 3404200.17it/s]

0it [00:00, ?it/s]

  0%|          | 0/4542 [00:00<?, ?it/s]/home/dave/.conda/envs/dvc/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729002806/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
1654784it [00:00, 2165530.86it/s]                             
8192it [00:00, 33762.4

**IMPORTANT:** Be sure to run the `git add` command above and also `git commit` before running experiments. Anytime you modify the pipeline, be sure to `dvc repro` and track changes with git before running experiments.

In [4]:
%%bash
git add dvc.lock data/.gitignore
git commit -m "download data"

[dev e16716b] download data
 1 file changed, 1 insertion(+), 1 deletion(-)


### Run an experiment

Run an experiment with the default parameters defined in `params.yaml`.

In [5]:
%%bash
dvc exp run

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'a79208f'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '7e16ced'.
Reproduced experiment '7e16ced'.


Review the output of the run, including identifying hashes, metrics, and parameters:

In [6]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment  ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace   │ -        │ 0.4902 │ 2.1588 │ 0.001 │ 0            │
│ dev         │ 10:40 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ 7e16ced │ 10:41 AM │ 0.4902 │ 2.1588 │ 0.001 │ 0            │
│ ├─╨ a79208f │ 10:41 AM │  0.151 │ 2.2506 │ 0.001 │ 0            │
└─────────────┴──────────┴────────┴────────┴───────┴──────────────┘


Note that two experiments were run. These are checkpoints for the run. It's not necessary to have checkpoints for experiments, but they can be helpful for models that may be run for a number of epochs. See below for more information about how checkpoints work.

### Experiment with different parameters

Experiments can be run and compared with different parameters.

In [7]:
%%bash
dvc exp run --params weight_decay=0.1

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '6cb12ea'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '2e1b5db'.
Reproduced experiment '2e1b5db'.


In [9]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment      ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace       │ -        │ 0.1933 │ 2.2893 │ 0.001 │ 0.1          │
│ checkpoint_only │ 03:32 PM │      - │      - │ 0.001 │ 0            │
│ │ ╓ 5ed8c14     │ 03:33 PM │ 0.1933 │ 2.2893 │ 0.001 │ 0.1          │
│ ├─╨ 6c1e6bd     │ 03:33 PM │  0.183 │ 2.2972 │ 0.001 │ 0.1          │
│ │ ╓ 5b61485     │ 03:32 PM │ 0.1292 │ 2.2936 │ 0.001 │ 0            │
│ ├─╨ 442c72c     │ 03:32 PM │  0.101 │ 2.2998 │ 0.001 │ 0            │
└─────────────────┴──────────┴────────┴────────┴───────┴──────────────┘


Increasing `weight_decay` helped a bit. Next, try different `lr` parameters.

Experiments can be added in bulk to the queue and executed on demand (see the `-j` flag for parallel execution!).

In [8]:
%%bash
dvc exp run --params lr=0.01 --queue
dvc exp run --params lr=0.1 --queue

Queued experiment 'a5461b5' for future execution.
Queued experiment 'a3ff998' for future execution.


In [9]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment   ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace    │ -        │ 0.1009 │ 2.3051 │ 0.001 │ 0.1          │
│ dev          │ 10:40 AM │      - │      - │ 0.001 │ 0            │
│ │ ╓ 2e1b5db  │ 10:42 AM │ 0.1009 │ 2.3051 │ 0.001 │ 0.1          │
│ ├─╨ 6cb12ea  │ 10:42 AM │ 0.1002 │ 2.3056 │ 0.001 │ 0.1          │
│ │ ╓ 7e16ced  │ 10:41 AM │ 0.4902 │ 2.1588 │ 0.001 │ 0            │
│ ├─╨ a79208f  │ 10:41 AM │  0.151 │ 2.2506 │ 0.001 │ 0            │
│ ├── *a3ff998 │ 10:42 AM │      - │      - │ 0.1   │ 0.1          │
│ └── *a5461b5 │ 10:42 AM │      - │      - │ 0.01  │ 0.1          │
└──────────────┴──────────┴────────┴────────┴───────┴──────────────┘


In [12]:
%%bash
dvc exp run --run-all

Stage '../../../../tmp/tmpkgcbkf4q/dvc.yaml:download' didn't change, skipping
Running stage '../../../../tmp/tmpkgcbkf4q/dvc.yaml:train' with command:
	python train.py


ERROR: Failed to reproduce experiment '928d58d' - Stage: '../../../../tmp/tmpkgcbkf4q/dvc.yaml:download'
ERROR: Error generating checkpoint, stage: '../../../../tmp/tmpkgcbkf4q/dvc.yaml:train' will be aborted - file path '/home/dave/Code/dvc-exp-mnist' is outside of DVC repo
ERROR: Failed to reproduce experiment '50deaf6' - [Errno 2] No such file or directory: '/tmp/tmpq74w6liy'


### Iteratively train using checkpoints

Use checkpoints to periodically save the model during training (as shown above), and to resume training from previously saved state. Resume training the experiment with the best accuracy.

In [13]:
%%bash
dvc exp show --sort-by acc

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment      ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace       │ -        │ 0.1933 │ 2.2893 │ 0.001 │ 0.1          │
│ checkpoint_only │ 03:32 PM │      - │      - │ 0.001 │ 0            │
│ │ ╓ 5b61485     │ 03:32 PM │ 0.1292 │ 2.2936 │ 0.001 │ 0            │
│ ├─╨ 442c72c     │ 03:32 PM │  0.101 │ 2.2998 │ 0.001 │ 0            │
│ │ ╓ 5ed8c14     │ 03:33 PM │ 0.1933 │ 2.2893 │ 0.001 │ 0.1          │
│ ├─╨ 6c1e6bd     │ 03:33 PM │  0.183 │ 2.2972 │ 0.001 │ 0.1          │
│ ├── *928d58d    │ 03:36 PM │      - │      - │ 0.1   │ 0.1          │
│ └── *50deaf6    │ 03:36 PM │      - │      - │ 0.01  │ 0.1          │
└─────────────────┴──────────┴────────┴────────┴───────┴──────────────┘


In [14]:
%%bash
dvc exp res -r 5ed8c14

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '7e5c6cd'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '0f01a80'.
Reproduced experiment '0f01a80'.


In [15]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment      ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace       │ -        │ 0.1536 │ 2.2662 │ 0.001 │ 0.1          │
│ checkpoint_only │ 03:32 PM │      - │      - │ 0.001 │ 0            │
│ │ ╓ 0f01a80     │ 03:37 PM │ 0.1536 │ 2.2662 │ 0.001 │ 0.1          │
│ │ ╟ 7e5c6cd     │ 03:36 PM │ 0.2337 │ 2.2796 │ 0.001 │ 0.1          │
│ │ ╟ 5ed8c14     │ 03:33 PM │ 0.1933 │ 2.2893 │ 0.001 │ 0.1          │
│ ├─╨ 6c1e6bd     │ 03:33 PM │  0.183 │ 2.2972 │ 0.001 │ 0.1          │
│ │ ╓ 5b61485     │ 03:32 PM │ 0.1292 │ 2.2936 │ 0.001 │ 0            │
│ ├─╨ 442c72c     │ 03:32 PM │  0.101 │ 2.2998 │ 0.001 │ 0            │
│ ├── *928d58d    │ 03:36 PM │      - │      - │ 0.1   │ 0.1          │
│ └── *50deaf6    │ 03:36 PM │      - │      - │ 0.01  │ 0.1          │
└─────────────────┴──────────┴────────┴────────┴───────┴────────

### Persist models

Additonal epochs didn't improve accuracy, so commit the model iteration with peak accuracy. Checkout the experiment rev in dvc and then commit to git.

In [16]:
%%bash
dvc exp checkout 7e5c6cd
cat metrics.yaml

acc: 0.1536
loss: 2.2661871910095215


ERROR: Experiment derived from '5ed8c14', expected '14d8c3b'.
